In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
training = fc.eightb.training.training_5M.NMSSM_XYY_YToHH_8b_MX_1000_MY_450

In [4]:
def get_train(path):
    return path.replace('ntuple.root','reweighted_ntuple_0.root')

In [5]:
training = Tree( get_train(training) )

100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


In [6]:
trees = ObjIter([training])

In [7]:
# (signal+bkg).apply(lambda t : build_collection(t, 'H\dY\d_b\d', 'jet', ordered='pt'), report=True)

t8btag = CollectionFilter('jet', filter=lambda t: ak_rank(-t.jet_btag, axis=-1) < 8)
trees = trees.apply(t8btag)

In [8]:
jet_p4 = build_p4(trees[0], 'jet', use_regressed=True)

In [33]:
jet_p4[:,:4]

<MomentumArray4D [[{pt: 1.05e+03, ... m: 8.71}]] type='603144 * var * Momentum4D...'>

In [189]:
n = 8

In [190]:
j1, j2 = ak.unzip(ak.combinations(jet_p4[:,:n], n=2))

In [191]:
hm = (j1 + j2).m/125

<Array [[9.34, 7.37, 6.69, ... 0.525, 0.386]] type='603144 * var * float32'>

In [192]:
i1, i2 = ak.unzip(ak.combinations(np.arange(n),2,axis=0))

In [193]:
M = np.zeros((n,n))

for i, j, m in zip(i1, i2, hm[0]):
    M[i,j] = m 
    M[j,i] = m

In [194]:
M.shape

(8, 8)

In [195]:
from sklearn.manifold import MDS 


mds = MDS(n_components=n, dissimilarity='precomputed')

In [196]:
X = mds.fit_transform(M)

In [197]:
x1, x2 = ak.unzip(ak.combinations(X, n=2, axis=0))
y = ak.sum( (x1-x2)**2,axis=-1 )

In [200]:
y, hm[0]

(<Array [66.4, 44.2, 46.4, ... 0.24, 1.15, 1.32] type='28 * float64'>,
 <Array [9.34, 7.37, 6.69, ... 1.26, 1.36] type='28 * float32'>)

In [177]:
np.sum((X[0]-X[1])**2), M[0,1]

(87.63769699430819, 9.34167766571045)